In [8]:
# from distance_request import *
from datetime import datetime
import googlemaps
import pandas as pd

In [9]:
# Read in call & ambulance files
calls_df = pd.read_csv('calls.csv', parse_dates=[3])
amb_df = pd.read_csv('ambulance_loc.csv')

# Select the most recent call and the available ambulances
current_call = calls_df.loc[calls_df.TIME == max(calls_df.TIME)]
available_amb = amb_df.loc[amb_df.AVAILABLE == 1]

In [10]:
available_amb.head()

,AMB_ID,LAT,LONG,AVAILABLE
0,amb_1,37.615223,-122.389977,1
1,amb_2,37.802400,-122.405800,1
2,amb_3,37.750238,-122.433703,1
4,amb_5,37.774929,-122.419416,1
6,amb_7,37.785413,-122.429383,1


In [11]:
current_call

,CALL_ID,LAT,LONG,TIME
0,call_1,37.788001,-122.407409,2017-12-01 23:05:00


In [12]:
# Get coordinates of most recent call and available ambulances
call_coord = [(current_call.LAT[0], current_call.LONG[0])]
amb_coord = [(row[2], row[3]) for row in available_amb.itertuples()]

# Use call time as proxy for departure time
dep_time = current_call.TIME[0]

In [20]:
# Get api key to make call
def get_api_key(filepath):
    with open(filepath) as f:
        content = f.readlines()
    # remove whitespace characters at the end of each line
    content = [x.strip() for x in content]
    key = content[0]
    return key
key = get_api_key('/Users/melaniecostello/Desktop/master_api_key.txt')
# key = get_api_key('/Users/melaniecostello/Desktop/maps_distance_key.txt')

In [21]:
# Make API call & store results in obj 'result'
gmaps = googlemaps.Client(key=key)
result = gmaps.distance_matrix(amb_coord, call_coord, mode="driving", units="imperial", departure_time=dep_time)

In [22]:
result

{'destination_addresses': ['350 Post St, San Francisco, CA 94108, USA'],
 'origin_addresses': ['International Terminal Arrivals Level, San Francisco, CA 94128, USA',
  '39-95 Telegraph Hill Blvd, San Francisco, CA 94133, USA',
  '1400-1414 Castro St, San Francisco, CA 94114, USA',
  '8411 Market St, San Francisco, CA 94103, USA',
  '1682 Post St, San Francisco, CA 94115, USA',
  '4900 Marie P DeBartolo Way, Santa Clara, CA 95054, USA',
  '6 Broadway, San Francisco, CA 94111, USA',
  '150 3rd St, San Francisco, CA 94103, USA'],
 'rows': [{'elements': [{'distance': {'text': '13.8 mi', 'value': 22205},
     'duration': {'text': '25 mins', 'value': 1510},
     'duration_in_traffic': {'text': '24 mins', 'value': 1416},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '1.6 mi', 'value': 2544},
     'duration': {'text': '14 mins', 'value': 857},
     'duration_in_traffic': {'text': '13 mins', 'value': 804},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '4.1 mi', '

In [38]:
# Helper function to parse results
def parse_api_obj(result, available_amb, current_call):
    output_mat = pd.DataFrame()
    for idx, row in enumerate(result['rows']):
        row_mat = pd.DataFrame()
        mat = row['elements'][0]
        for key, val in mat.items():
            if key != 'status':
                df = pd.DataFrame.from_dict(val, orient='index')
                df = df.transpose()
                df.columns = [key + "_" + c for c in df.columns]
                if row_mat.empty:
                    row_mat = df
                else:
                    row_mat = pd.concat([row_mat, df], axis=1)
        if output_mat.empty:
            output_mat = row_mat
        else:
            output_mat = output_mat.append(row_mat)
#     output_mat.index = [amb for amb in available_amb.AMB_ID]
    output_mat['ambulance'] = [amb for amb in available_amb.AMB_ID]
    output_mat['amb_lat'] = [lat for lat in available_amb.LAT]
    output_mat['amb_long'] = [long for long in available_amb.LONG]
    output_mat['call_lat'] = current_call.LAT[0]
    output_mat['call_long'] = current_call.LONG[0]
    output_mat.reset_index(inplace=True, drop=True)
    return output_mat

In [39]:
# Get data frame of available ambulances
options = parse_api_obj(result, available_amb, current_call)

In [40]:
options

,distance_text,distance_value,duration_in_traffic_text,duration_in_traffic_value,duration_text,duration_value,ambulance,amb_lat,amb_long,call_lat,call_long
0,13.8 mi,22205,24 mins,1413,25 mins,1496,amb_1,37.615223,-122.389977,37.788001,-122.407409
1,1.6 mi,2544,13 mins,801,14 mins,851,amb_2,37.802400,-122.405800,37.788001,-122.407409
2,4.1 mi,6520,21 mins,1258,23 mins,1359,amb_3,37.750238,-122.433703,37.788001,-122.407409
3,1.8 mi,2934,11 mins,636,11 mins,674,amb_5,37.774929,-122.419416,37.788001,-122.407409
4,1.2 mi,1951,8 mins,477,8 mins,494,amb_7,37.785413,-122.429383,37.788001,-122.407409
5,44.1 mi,70986,52 mins,3144,57 mins,3400,amb_8,37.403204,-121.969841,37.788001,-122.407409
6,1.4 mi,2243,9 mins,564,10 mins,627,amb_9,37.799263,-122.397673,37.788001,-122.407409
7,0.6 mi,953,5 mins,277,5 mins,308,amb_10,37.785718,-122.401051,37.788001,-122.407409


In [155]:
# Select ambulance for response based on desired metric
chosen = options.loc[options.duration_in_traffic_value == min(options.duration_in_traffic_value)].index[0]

In [156]:
chosen

'amb_10'

# For parsing UI input - geocoding an address

In [46]:
# Get API key (same function as used above)
def get_api_key(filepath):
    with open(filepath) as f:
        content = f.readlines()
    # remove whitespace characters at the end of each line
    content = [x.strip() for x in content]
    key = content[0]
    return key

key = get_api_key('/Users/melaniecostello/Desktop/master_api_key.txt')

def get_coordinates(key, addr):
    """Runs google maps geocoding api to return lat/long coords
    for a list of addresses.
    key: string (API key)
    addr: list of strings (addresses)"""
    gmaps = googlemaps.Client(key=key)
    coords = []
    for ad in addr:
        geocode_result = gmaps.geocode(ad)
        lat_long = geocode_result[0]['geometry']['location']
        # Add tuple with lat & long to coords output
        coords.append((lat_long['lat'], lat_long['lng']))
    return coords
        
get_coordinates(key, ['NW 4th & Couch', '2500 NW Crosswater Ter, Beaverton OR', '4065 Paradise Valley Rd, Chinook MT'])

[(45.5238131, -122.6744172),
 (45.53792019999999, -122.8526896),
 (48.5611042, -109.1127062)]